In [122]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from random import gauss
import warnings
warnings.filterwarnings('ignore')

In [123]:
with open('/Users/Dovla/Downloads/quotedata.dat','r') as f:
    underlying = float(f.readline().split(',')[1])
dat = pd.read_csv('/Users/Dovla/Downloads/quotedata.dat', sep=",", skiprows=2)
calls = dat.iloc[:,0:7]
calls['Y'], calls['M'],calls['D'],calls['K'],calls['Code'] = calls['Calls'].str.split(' ', -1).str
calls['Code'],calls['Exch'] = calls['Code'].str.split('-', -1).str
calls['K'] = calls.K.astype(float)
calls['Time'] = pd.to_datetime(calls.Y+calls.M+calls.D,format='%Y%b%d')
calls['Time1'] = calls['Time'] - calls['Time'][0]
calls['Ti'] = calls['Time1'].dt.days / 365
calls['S'] = underlying
calls['r'] = 0.03
calls['opMidPrice'] = (calls['Bid'] + calls['Ask'])/2
calls['sigma'] = 0.1
calls = calls[calls['Exch'].isnull()]
calls1 = calls[['S','K','Ti','r','sigma','opMidPrice']]

In [124]:
def d1(x):
    d1 = (np.log(x.S/x.K) + (0.03 + x.sigma**2/2)*x.Ti) / (x.sigma * np.sqrt(x.Ti))
    return d1
def d2(x):
    d2 = x.d1 - x.sigma * np.sqrt(x.Ti)
    return d2
def call(x):
    c0 =  x.S * norm.cdf(x.d1) - x.K * exp(-0.03*(x.Ti)) * norm.cdf(x.d2)
    return c0

In [125]:
calls1['d1'] = calls1.apply(d1, axis=1)
calls1['d2'] = calls1.apply(d2, axis=1)
calls1['optBSprice'] = calls1.apply(call, axis=1)

In [126]:
def GBM(S,sigma,r,T):
    return S * exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * gauss(0,1.0))
def MC(x, sims=1000):
    payoffs = []
    for i in range(sims):
        S_T = GBM(x.S,x.sigma,x.r,x.Ti)
        payoffs.append(max(0.0,S_T-x.K))
    price = np.exp(-x.r * (x.Ti)) * (sum(payoffs) / float(sims))
    return price
calls1['CallMC'] = calls1.apply(MC, axis=1)